<img src="images/hdd.jpg" width="20%" align="right">

DataFrame Storage
================

Efficient storage can dramatically improve performance, particularly when operating repeatedly from disk.

Decompressing text and parsing CSV files is expensive.  One of the most effective strategies with medium data is to use a binary storage format like HDF5.  Often the performance gains from doing this is sufficient so that you can switch back to using Pandas again instead of using `dask.dataframe`.

In this section we'll learn how to efficiently arrange and store your datasets in on-disk binary formats.  We'll use the following:

1.  [Pandas `HDFStore`](http://pandas.pydata.org/pandas-docs/stable/io.html#io-hdf5) format on top of `HDF5`
2.  Categoricals for storing text data numerically

**Main Take-aways**

1.  Storage formats affect performance by an order of magnitude
2.  Text data will keep even a fast format like HDF5 slow
3.  A combination of binary formats, column storage, and partitioned data turns one second wait times into 80ms wait times.

## Note!

If you're on a Mac you need to install the previous versions of `h5py` and `pytables`, like so:

```sh
conda install h5py=2.4.0 pytables=3.1.1
```

The most recent versions of these libraries cause a segfault when writing to HDF5 storage.

### Setup

Create data if we don't have any

In [ ]:
from prep import accounts_csvs
accounts_csvs(3, 1000000, 500)

### Read CSV

First we read our csv data as before

In [ ]:
import os
filename = os.path.join('data', 'accounts.*.csv')
filename

In [ ]:
import dask.dataframe as dd
df_csv = dd.read_csv(filename)
df_csv.head()

### Write to HDF5

Pandas contains a specialized HDF5 format, `HDFStore`.  The ``dd.DataFrame.to_hdf`` method works exactly like the ``pd.DataFrame.to_hdf`` method.

In [ ]:
target = os.path.join('data', 'accounts.h5')
target

In [ ]:
%time df_csv.to_hdf(target, '/data')

In [ ]:
df_hdf = dd.read_hdf(target, '/data')
df_hdf.head()

### Compare CSV to HDF5 speeds

We do a simple computation that requires reading a column of our dataset and compare performance between CSV files and our newly created HDF5 file.  Which do you expect to be faster?

In [ ]:
%time df_csv.amount.sum().compute()

In [ ]:
%time df_hdf.amount.sum().compute()

Sadly they are about the same.  

The culprit here is `names` column, which is of `object` dtype and thus hard to store efficiently.  There are two problems here:

1.  How do we store text data like `names` efficiently on disk?
2.  Why did we have to read the `names` column when all we wanted was `amount`

### 1.  Store text efficiently with categoricals

We can use Pandas categoricals to replace our object dtypes with a numerical representation.  This takes a bit more time up front, but results in better performance.

More on categoricals at the [pandas docs](http://pandas.pydata.org/pandas-docs/stable/categorical.html) and [this blogpost](http://matthewrocklin.com/blog/work/2015/06/18/Categoricals).

In [ ]:
# Categorize data, then store in HDFStore
%time df_hdf.categorize(columns=['names']).to_hdf(target, '/data2')

In [ ]:
# It looks the same
df_hdf = dd.read_hdf(target, '/data2')
df_hdf.head()

In [ ]:
# But loads more quickly
%time df_hdf.amount.sum().compute()

This is significantly faster.  This tells us that it's not only the file type that we use but also how we represent our variables that influences storage performance.

However this can still be better.  We had to read all of the columns (`names` and `amount`) in order to compute the sum of one (`amount`).  We'll improve further on this with `castra`, an on-disk column-store.  First though we learn about how to set an index in a dask.dataframe.

### Exercise

Index your dataframe by `id`, then store it into a `Castra` file as we do above.  Inspect your data and verify that the index column is `id` and that it starts at `0`.

What are the divisions of your dataframe?

Use the `.loc` accessor to get the records corresponding to account `100`.  Compare this to the speed you saw in the last section.

## Remote files

Dask can access Amazon S3 buckets or on HDFS

Advantages:
* scalable, secure storage

Disadvantages:
* network speed becomes bottleneck
* only utilize local compute resources
    * See dask.distributed



For this we'll need s3fs.

```
conda install s3fs
```

```python
taxi = dd.read_csv('s3://nyc-tlc/trip data/yellow_tripdata_2015-*.csv')
```

**Warning**: operations over the Internet can take a long time to run.

## Parquet

TBD